# Pluri-Gaussian 2D

## 0. Preamble

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    // Remove Scrollbar in outputs
    return false;
}

This tutorial requires to install

- gstlearn
- numpy
- pandas
- matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import os

## 1. Prepare the basic gstlearn objects

Initial objects are located in a specific Directory which is defined in the next Container operation. Note that his operation will modify automatically all the names of the Files retreived using Serialize / Deserialize operation (not when reading using CSV). Also note that the Container name must be ended using a "/" (as its serves as a Directory).

In [ ]:
ndim = 2
gl.ASpaceObject.defineDefaultSpace(gl.SPACE_RN,ndim)

### Load the data file

This Load is performed starting from a CSV file.

In [ ]:
dataDir = os.path.join(os.path.pardir,os.path.pardir,"doc","data","BRGM")
datCat = pd.read_csv(os.path.join(dataDir,"Nitrates_LANU.csv"),sep=";")
datCat.head()

### Loading polygon from a file

The polygon is created by deserializing the Neutral Polygon File (located in data Directory)

In [ ]:
poly = gl.Polygons.createFromNF(os.path.join(dataDir,"poly_LANU.ascii"))
poly

### Creation of the gstlearn data base 

In [ ]:
dat = gl.Db()
fields = ["X","Y","LANU"]
dat[fields] = datCat[fields].values

###  Specification of the role of each variable (named "locators" in gstlearn)

In [ ]:
dat.setLocators(["X","Y"],gl.ELoc.X) #Coordinates
dat.setLocator("LANU",gl.ELoc.Z) #Variable of interest
dat

### Creation of the output grid

The output grid will contain 47 x 101 nodes. It is built to cover the data file plus an extension of 10000 x 10000.

In [ ]:
Result = gl.DbGrid()
Result.resetCoveringDb(dat,[47,101],[],[],[50000,50000])

### Add a selection (mask the cells outside the polygon)

In [ ]:
gl.db_polygon(Result,poly)
ax = gp.grid(Result,name="Polygon*",usesel=False,flagColorBar=False,figsize=[10,10],
            title="Initial information")
ax = gp.point(dat,"LANU",size=2,ax=ax)
ax = gp.polygon(poly,end_plot=True,linewidth=1,edgeColor="r",ax=ax)

## 2. Computation of the proportions

### 2.1 Compute global proportions (for information)

In [ ]:
propGlob = gl.dbStatisticsFacies(dat)
ncat = len(propGlob)
for i in range(ncat):
    print("Proportion of facies "+str(i+1),"=",propGlob[i])

### 2.2 Compute local proportions

The next parts will be simplified in a future dedicated API

**2.2.1 Creation of the spatial regularization model for proportions**

In [ ]:
model = gl.Model.createFromDb(Result)
cova = gl.CovAniso(gl.ECov.BESSEL_K,50000.,2.,1.,model.getContext()) 
model.addCova(cova)

In [ ]:
err = gl.db_proportion_estimate(dat,Result,model)

In [ ]:
Result.display()

In [ ]:
dbfmt = gl.DbStringFormat()
dbfmt.setParams(gl.FLAG_STATS)
dbfmt.setNames(["Prop.*"])
Result.display(dbfmt)

### Display the results

In [ ]:
for i in range(ncat):
    ax = gp.grid(Result,name="Prop."+str(i+1),end_plot=False,title="Proportion Facies #"+str(i+1),
                figsize=[10,10])
    gp.point(dat,size=0.5,color="black",ax=ax)
    dat.addSelectionByLimit("LANU",gl.Limits((i+1,i+1)),"SelPoint")
    gp.point(dat,size=0.8,ax=ax,color="red")
    dat.deleteColumn("SelPoint")
    gp.polygon(poly,end_plot=True,linewidth=1,edgeColor="r",ax=ax)

Creating the environment to infer the Rule. It uses a variogram calculated over very few lags close to the origin.

In [ ]:
nlag = 2
varioParam = gl.VarioParam()
dirparam = gl.DirParam(ndim, nlag, 100);
varioParam.addDirs(dirparam);
ruleprop = gl.RuleProp.createFromDb(Result);         
ruleprop.fit(dat, varioParam, 1);
ngrf = ruleprop.getRule().getGRFNumber()
print("Number of GRF =",ngrf)

In [ ]:
ax=gp.rule(ruleprop.getRule())

In [ ]:
nlag = 19;
dirparam = gl.DirParam(ndim, nlag, 1000);
covparam = gl.VarioParam();
covparam.addDirs(dirparam);
cov = gl.variogram_pgs(dat,covparam,ruleprop);

In [ ]:
cov.display()

We extract the experimental variograms of each GRF.

In [ ]:
vario1 = gl.Vario(cov)
vario1.reduce([0],gl.VectorInt(),True)
if ngrf > 1:
    vario2 = gl.Vario(cov)
    vario2.reduce([1],gl.VectorInt(),True)

In [ ]:
vario1.display()
if ngrf > 1:
    vario2.display()

We now fit the model of each GRF considered as independent. The fit is performed under the constraint that the sill should be 1.

In [ ]:
ctxt = gl.CovContext(1,2) # use default space
option = gl.Option_AutoFit()
option.setConstantSillValue(1.)
covs  = [gl.ECov.BESSEL_K, gl.ECov.EXPONENTIAL]

modelPGS1 = gl.Model(ctxt)
modelPGS1.fit(vario1,covs,False,option)
modelPGS1.display()

if ngrf > 1:
    modelPGS2 = gl.Model(ctxt)
    modelPGS2.fit(vario2,covs,False,option)
    modelPGS2.display()
else:
    modelPGS2 = None

For each GRF, we can plot the experimental variogram as well as the fitted model.

In [ ]:
vario1.getVar(0,0)

In [ ]:
ax=gp.varmod(vario1,modelPGS1)
if ngrf > 1:
    ax=gp.varmod(vario2,modelPGS2)

In this paragraph, we compare the experimental indicator variogram to the one derived from the Model of the underlying GRFs.

In [ ]:
dirparamindic = gl.DirParam(ndim, nlag, 1000)
varioparamindic = gl.VarioParam()
varioparamindic.addDirs(dirparamindic)
varioindic = gl.Vario(varioparamindic,dat)
err = varioindic.computeIndic()

In [ ]:
varioindic2 = gl.model_pgs(dat, varioparamindic, ruleprop, modelPGS1, modelPGS2);

In [ ]:
axs=gp.varmod(varioindic,figsize=[10,10],linestyle='solid')
axs=gp.varmod(varioindic2,axs=axs,linestyle='dashed')

In [ ]:
neigh = gl.NeighUnique.create(2)
err = gl.simpgs(dat,Result,ruleprop,modelPGS1,modelPGS2,neigh)

In [ ]:
ax = gp.grid(Result,figsize=[10,10])